In [1]:
import librosa
import os

In [2]:
import mir_eval

In [3]:
from collections import OrderedDict

In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
pd.set_option('precision', 4, "display.max_rows", 999)

In [5]:
def make_beatles_corpus(iso_path):
    
    # Beat files
    beat_path = os.path.join(iso_path, 'beat')
    
    annotations = librosa.util.find_files(beat_path, ext='txt')
    
    audio = [ann.replace('/beat/', '/audio/').replace('.txt', '.flac') for ann in annotations]
    
    
    data = []
    for aud, ann in zip(audio, annotations):
        if os.path.exists(aud) and os.path.exists(ann):
            data.append((aud, ann))
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [6]:
def make_smc_data(smc_path):
    
    # Beat files
    beat_path = os.path.join(smc_path, 'SMC_MIREX_Annotations')
    annotations = librosa.util.find_files(beat_path, ext='txt')
    
    # Audio files
    audio_path = os.path.join(smc_path, 'SMC_MIREX_Audio')
    audio = librosa.util.find_files(audio_path, ext='wav')
    
    data = zip(audio, annotations)
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [7]:
def make_output_path(base, outpath):
    
    root = os.path.splitext(base)[0]
    
    output = os.path.join(outpath, os.path.extsep.join([root, 'json']))
    
    return output

In [8]:
def analyze(dframe, outpath='/home/bmcfee/git/librosa_parameters/data/'):
    
    index = dframe.index[0]
    
    base = os.path.basename(dframe['audio'][index])
    
    outfile = make_output_path(base, outpath)
    
    if os.path.exists(outfile):
        print 'Cached {}'.format(base)
        data = pd.read_json(outfile, orient='records')
        return data
    else:
        print 'Processing {}'.format(base)
    
    # Load the truth
    ref_times = pd.read_table(dframe['annotation'][index], header=None, sep='\s+',
                              usecols=[0], error_bad_lines=False)[0].values

    # Load the audio
    sr = 22050
    y, _ = librosa.load(dframe['audio'][index], None)
    y = librosa.resample(y, _, sr, res_type='sinc_fastest')
    
    # Construct the output container
    results = []
    
    # Onset strength parameters
    
    for fmax in [8000, None]:
        for n_mels in [32, 64, 128]:
            S = librosa.feature.melspectrogram(y=y, sr=sr, fmax=fmax, n_mels=n_mels)
            
            for aggregate in [np.mean, np.median]:
                # Compute the onset detection function
                oenv = librosa.onset.onset_strength(S=S, sr=sr,
                                                    aggregate=aggregate)
                
                    # Tempo estimator parameters
                for ac_size in [2, 4, 8]:
                    for std_bpm in [0.5, 1.0, 2.0]:
                        tempo = librosa.beat.estimate_tempo(oenv,
                                                                    sr=sr,
                                                                    ac_size=ac_size,
                                                                    std_bpm=std_bpm)
                                
                        for tightness in [50, 100, 500]:
                            # Evaluate the predictions
                            params = {'aggregate': aggregate.__name__,
                                              'fmax': fmax,
                                              'n_mels': n_mels,
                                              'ac_size': ac_size,
                                              'std_bpm': std_bpm,
                                              'tightness': tightness}
                            
                            _, beats = librosa.beat.beat_track(sr=sr,
                                                                       onset_envelope=oenv,
                                                                       trim=False, 
                                                                       tightness=tightness,
                                                                       bpm=tempo)
                                    
                            est_times = librosa.frames_to_time(beats, sr=sr)
                            scores = mir_eval.beat.evaluate(ref_times, est_times)
                            
                            cont = OrderedDict(index=index)
                            cont.update(params)
                            cont.update(scores)
                            results.append(cont)
    # Blow away the cache
    #librosa.cache.clear()
    data = pd.DataFrame.from_dict(results, orient='columns')
    data.to_json(outfile, orient='records')
        
    return data

In [9]:
def analyze_corpus(corpus):
    
    results = None
    for idx in corpus.index:
        new_results = analyze(corpus.loc[[idx]])
        if results is None:
            results = new_results
        else:
            results = pd.concat([results, new_results])
            
    return results

In [10]:
from joblib import Parallel, delayed

In [11]:
def p_analyze_corpus(corpus, n_jobs=3):
    
    results = None
    
    dfunc = delayed(analyze)
    
    results = Parallel(n_jobs=n_jobs, verbose=10)(dfunc(corpus.loc[[idx]])
                                                  for idx in corpus.index)

    return pd.concat(results)

---

In [12]:
smc_data = make_smc_data('/home/bmcfee/data/SMC_Mirex/')

In [13]:
smc_results = analyze_corpus(smc_data)

Processing SMC_001.wav
Processing SMC_002.wav


Processing SMC_003.wav


Processing SMC_004.wav


Processing SMC_005.wav


Processing SMC_006.wav

...


Processing SMC_285.wav


Processing SMC_286.wav


Processing SMC_287.wav


Processing SMC_288.wav


Processing SMC_289.wav

In [15]:
#smc_results.to_json('/home/bmcfee/git/librosa_parameters/smc_beat_results.json', orient='records')

In [19]:
smc_results = pd.read_json('/home/bmcfee/git/librosa_parameters/smc_beat_results.json', orient='records')

In [20]:
smc_scores = smc_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [21]:
best_igain = smc_scores['Information gain'].argmax()

In [22]:
best_fmeas = smc_scores['F-measure'].argmax()

In [23]:
best_amlt = smc_scores['Any Metric Level Total'].argmax()

In [29]:
smc_scores.loc[[best_igain, best_fmeas, best_amlt]]

,,,,,,Any Metric Level Continuous,Any Metric Level Total,Cemgil,Cemgil Best Metric Level,Correct Metric Level Continuous,Correct Metric Level Total,F-measure,Goto,Information gain,P-score,index
aggregate,fmax,n_mels,ac_size,std_bpm,tightness,,,,,,,,,,,
median,8000,128,8,2,100,0.172,0.314,0.237,0.305,0.105,0.172,0.353,0.078,0.176,0.479,108
mean,8000,128,2,1,10,0.079,0.261,0.251,0.322,0.034,0.092,0.368,0.009,0.142,0.458,108
median,8000,128,8,1,50,0.154,0.334,0.243,0.303,0.097,0.173,0.361,0.078,0.174,0.493,108


In [30]:
smc_scores.loc[best_igain]

Any Metric Level Continuous          0.172
Any Metric Level Total               0.314
Cemgil                               0.237
Cemgil Best Metric Level             0.305
Correct Metric Level Continuous      0.105
Correct Metric Level Total           0.172
F-measure                            0.353
Goto                                 0.078
Information gain                     0.176
P-score                              0.479
index                              108.000
Name: (median, 8000.0, 128, 8, 2.0, 100), dtype: float64

In [31]:
smc_scores.loc[best_fmeas]

Any Metric Level Continuous          0.079
Any Metric Level Total               0.261
Cemgil                               0.251
Cemgil Best Metric Level             0.322
Correct Metric Level Continuous      0.034
Correct Metric Level Total           0.092
F-measure                            0.368
Goto                                 0.009
Information gain                     0.142
P-score                              0.458
index                              108.000
Name: (mean, 8000.0, 128, 2, 1.0, 10), dtype: float64

In [32]:
smc_scores.loc[best_amlt]

Any Metric Level Continuous          0.154
Any Metric Level Total               0.334
Cemgil                               0.243
Cemgil Best Metric Level             0.303
Correct Metric Level Continuous      0.097
Correct Metric Level Total           0.173
F-measure                            0.361
Goto                                 0.078
Information gain                     0.174
P-score                              0.493
index                              108.000
Name: (median, 8000.0, 128, 8, 1.0, 50), dtype: float64

---

In [12]:
beatles_data = make_beatles_corpus('/home/bmcfee/data/beatles_iso/')

In [13]:
beatles_results = p_analyze_corpus(beatles_data)

[Parallel(n_jobs=3)]: Done   1 out of  21 | elapsed:    0.0s remaining:    0.5s
[Parallel(n_jobs=3)]: Done   5 out of  31 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=3)]: Done   3 out of  33 | elapsed:    0.0s remaining:    0.5s
[Parallel(n_jobs=3)]: Done   8 out of  38 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=3)]: Done   7 out of  40 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  17 out of  71 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  26 out of 128 | elapsed:    0.2s remaining:    0.6s
[Parallel(n_jobs=3)]: Done  33 out of 159 | elapsed:    0.2s remaining:    0.7s
[Parallel(n_jobs=3)]: Done  37 out of 179 | elapsed:    0.2s remaining:    0.8s
[Parallel(n_jobs=3)]: Done  55 out of 179 | elapsed: 14.0min remaining: 31.5min
[Parallel(n_jobs=3)]: Done  73 out of 179 | elapsed: 29.2min remaining: 42.4min
[Parallel(n_jobs=3)]: Done  91 out of 179 | elapsed: 44.9min remaining: 43.4min
[Parallel(n_jobs=3)]: Done 109 out of 17

Cached 01_-_I_Saw_Her_Standing_There.flac
Cached 02_-_Misery.flac
Cached 03_-_Anna_(Go_To_Him).flac
Cached 04_-_Chains.flac
Cached 06_-_Ask_Me_Why.flac
Cached 08_-_Love_Me_Do.flac
Cached 05_-_Boys.flac
Cached 09_-_P._S._I_Love_You.flac
Cached 10_-_Baby_It's_You.flac
Cached 07_-_Please_Please_Me.flac
Cached 12_-_A_Taste_Of_Honey.flac
Cached 13_-_There's_A_Place.flac
Cached 11_-_Do_You_Want_To_Know_A_Secret.flac
Cached 14_-_Twist_And_Shout.flac
Cached 01_-_It_Won't_Be_Long.flac
Cached 02_-_All_I've_Got_To_Do.flac
Cached 03_-_All_My_Loving.flac
Cached 04_-_Don't_Bother_Me.flac
Cached 05_-_Little_Child.flac
Cached 06_-_Till_There_Was_You.flac
Cached 07_-_Please_Mister_Postman.flac
Cached 08_-_Roll_Over_Beethoven.flac
Cached 09_-_Hold_Me_Tight.flac
Cached 10_-_You_Really_Got_A_Hold_On_Me.flac
Cached 11_-_I_Wanna_Be_Your_Man.flac
Cached 12_-_Devil_In_Her_Heart.flac
Cached 14_-_Money.flac
Cached 13_-_Not_A_Second_Time.flac
Cached 01_-_A_Hard_Day's_Night.flac
Cached 03_-_If_I_Fell.flac
Cached 

In [14]:
beatles_results.to_json('/home/bmcfee/git/librosa_parameters/beatles_beat_results.json', orient='records')

In [15]:
beatles_scores = beatles_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [28]:
beatles_scores

Any Metric Level Continuous  \
aggregate fmax n_mels ac_size std_bpm tightness                                
mean      8000 32     2       0.5     50                               0.309   
                                      100                              0.390   
                                      500                              0.462   
                              1.0     50                               0.340   
                                      100                              0.431   
                                      500                              0.523   
                              2.0     50                               0.336   
                                      100                              0.422   
                                      500                              0.524   
                      4       0.5     50                               0.309   
                                      100                              0.390   
                                      500                              0.462   
                              1.0     50                               0.340   
                                      100                              0.431   
                                      500                              0.523   
                              2.0     50                               0.336   
                                      100                              0.422   
                                      500                              0.524   
                      8       0.5     50                               0.309   
                                      100                              0.390   
                                      500                              0.462   
                              1.0     50                               0.340   
                                      100                              0.431   
                                      500                              0.523   
                              2.0     50                               0.336   
                                      100                              0.422   
                                      500                              0.524   
               64     2       0.5     50                               0.291   
                                      100                              0.366   
                                      500                              0.429   
                              1.0     50                               0.340   
                                      100                              0.435   
                                      500                              0.520   
                              2.0     50                               0.328   
                                      100                              0.427   
                                      500                              0.509   
                      4       0.5     50                               0.291   
                                      100                              0.366   
                                      500                              0.429   
                              1.0     50                               0.340   
                                      100                              0.435   
                                      500                              0.520   
                              2.0     50                               0.328   
                                      100                              0.427   
                                      500                              0.509   
                      8       0.5     50                               0.291   
                                      100                              0.366   
                                      500                              0.429   
                              1.0     50         

In [17]:
best_igain_b = beatles_scores['Information gain'].argmax()
best_fmeas_b = beatles_scores['F-measure'].argmax()
best_amlt_b = beatles_scores['Any Metric Level Total'].argmax()

In [27]:
print best_igain
print best_fmeas
print best_amlt
print
print best_igain_b
print best_fmeas_b
print best_amlt_b

(u'median', 8000.0, 128, 8, 2.0, 100)
(u'mean', 8000.0, 128, 2, 1.0, 10)
(u'median', 8000.0, 128, 8, 1.0, 50)

(u'median', 8000.0, 128, 2, 1.0, 100)
(u'median', 8000.0, 128, 2, 0.5, 100)
(u'median', 8000.0, 128, 2, 1.0, 100)


In [42]:
good_scores = ['Any Metric Level Total', 'Information gain', 'F-measure']

In [43]:
smc_scores.loc[[best_igain, best_amlt]][good_scores] 

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    8       2       100                         0.314   
                              1       50                          0.334   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
median    8000 128    8       2       100                   0.176      0.353  
                              1       50                    0.174      0.361

In [44]:
smc_scores.loc[[best_igain_b, best_amlt_b]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    2       1       100                         0.327   
                                      100                         0.327   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
median    8000 128    2       1       100                   0.172      0.356  
                                      100                   0.172      0.356

In [45]:
beatles_scores.loc[[best_igain, best_amlt]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    8       2       100                         0.762   
                              1       50                          0.781   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
median    8000 128    8       2       100                   0.479      0.598  
                              1       50                    0.482      0.663

In [46]:
beatles_scores.loc[[best_igain_b, best_amlt_b]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
median    8000 128    2       1       100                         0.787   
                                      100                         0.787   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
median    8000 128    2       1       100                   0.493      0.666  
                                      100                   0.493      0.666

In [53]:
beatles_results['fmax'].unique()

array([ 8000.,    nan])

In [87]:
smc_scores.loc[[('mean', 8000, 128, 4, 1.0, 500), best_igain, best_igain_b]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
mean      8000 128    4       1       500                         0.284   
median    8000 128    8       2       100                         0.314   
                      2       1       100                         0.327   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
mean      8000 128    4       1       500                   0.142      0.311  
median    8000 128    8       2       100                   0.176      0.353  
                      2       1       100                   0.172      0.356

In [88]:
beatles_scores.loc[[(u'mean', 8000.0, 128, 4, 1.0, 500), best_igain, best_igain_b]][good_scores]

Any Metric Level Total  \
aggregate fmax n_mels ac_size std_bpm tightness                           
mean      8000 128    4       1       500                         0.727   
median    8000 128    8       2       100                         0.762   
                      2       1       100                         0.787   

                                                 Information gain  F-measure  
aggregate fmax n_mels ac_size std_bpm tightness                               
mean      8000 128    4       1       500                   0.433      0.670  
median    8000 128    8       2       100                   0.479      0.598  
                      2       1       100                   0.493      0.666

In [47]:
best_igain_b

(u'median', 8000.0, 128, 2, 1.0, 100)